In [1]:
import pandas as pd
import sqlite3

In [2]:
db = sqlite3.connect("c2_database.sqlite")

In [3]:
def run_query(query):
    return pd.read_sql_query(query, db)

In [4]:
run_query("SELECT tbl_name FROM sqlite_master;")

,tbl_name
0,station
1,status
2,trip
3,weather


In [17]:
run_query("SELECT * FROM trip LIMIT 5;")

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4100,1392,8/29/2013 9:57,South Van Ness at Market,66,8/29/2013 10:20,San Francisco Caltrain 2 (330 Townsend),69,319,Subscriber,94118
1,4133,1965,8/29/2013 10:17,San Francisco Caltrain 2 (330 Townsend),69,8/29/2013 10:50,Powell Street BART,39,316,Customer,
2,4213,931,8/29/2013 11:14,Grant Avenue at Columbus Avenue,73,8/29/2013 11:30,Post at Kearney,47,473,Customer,20002
3,4282,572,8/29/2013 11:54,San Francisco Caltrain (Townsend at 4th),70,8/29/2013 12:04,Townsend at 7th,65,538,Customer,37206
4,4308,1000,8/29/2013 12:04,Redwood City Caltrain Station,22,8/29/2013 12:21,Redwood City Caltrain Station,22,159,Subscriber,94061


In [6]:
run_query("SELECT start_station_name, end_station_name, duration FROM trip LIMIT 5;")

,start_station_name,end_station_name,duration
0,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1392
1,San Francisco Caltrain 2 (330 Townsend),Powell Street BART,1965
2,Grant Avenue at Columbus Avenue,Post at Kearney,931
3,San Francisco Caltrain (Townsend at 4th),Townsend at 7th,572
4,Redwood City Caltrain Station,Redwood City Caltrain Station,1000


In [7]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip
WHERE start_station_name = 'Paseo de San Antonio'
LIMIT 5;
'''

In [8]:
run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,Paseo de San Antonio,539
1,Paseo de San Antonio,St James Park,5334
2,Paseo de San Antonio,Paseo de San Antonio,18009
3,Paseo de San Antonio,San Salvador at 1st,226
4,Paseo de San Antonio,San Salvador at 1st,256


In [9]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip
WHERE duration<100
LIMIT 5;
'''

run_query(query)

,start_station_name,end_station_name,duration
0,Market at Sansome,Market at Sansome,75
1,Market at 10th,South Van Ness at Market,94
2,Powell at Post (Union Square),Powell at Post (Union Square),82
3,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building),60
4,Townsend at 7th,Townsend at 7th,83


In [10]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip
WHERE (duration<100) AND (start_station_name = 'Market at 10th')
LIMIT 5;
'''

run_query(query)

,start_station_name,end_station_name,duration
0,Market at 10th,South Van Ness at Market,94
1,Market at 10th,Market at 10th,91
2,Market at 10th,Market at 10th,80
3,Market at 10th,Market at 10th,74
4,Market at 10th,Market at 10th,69


In [11]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip
WHERE start_station_name = 'Paseo de San Antonio'
ORDER BY duration DESC
LIMIT 5;
'''

run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,MLK Library,167156
1,Paseo de San Antonio,Paseo de San Antonio,70017
2,Paseo de San Antonio,San Salvador at 1st,53153
3,Paseo de San Antonio,Paseo de San Antonio,18009
4,Paseo de San Antonio,Paseo de San Antonio,11519


In [12]:
run_query("SELECT AVG(duration) AS 'Average Duration' FROM trip;")

,Average Duration
0,1070.969428


In [13]:
query = '''
SELECT subscription_type, AVG(duration) AS 'Average Duration' FROM trip
GROUP BY subscription_type;
'''

run_query(query)

,subscription_type,Average Duration
0,Customer,3715.673978
1,Subscriber,580.480241


In [14]:
query = '''
SELECT station.city AS "City", COUNT(*) AS "Count"
FROM trip
INNER JOIN station
ON trip.start_station_name = station.name
GROUP BY station.city;
'''

run_query(query)

,City,Count
0,Mountain View,2232
1,Palo Alto,853
2,Redwood City,409
3,San Francisco,74304
4,San Jose,4748


In [20]:
query = '''
SELECT start.city AS "Start City", end.city AS "End City", trip.duration
FROM trip
INNER JOIN station AS start
ON trip.start_station_name = start.name
INNER JOIN station AS end
ON trip.end_station_name = end.name
WHERE start.city != end.city
ORDER BY duration DESC
LIMIT 1;
'''

run_query(query)

,Start City,End City,duration
0,Mountain View,San Francisco,179330


In [21]:
query = '''
SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney';
'''

run_query(query)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,17227,140,9/10/2013 20:37,Post at Kearney,47,9/10/2013 20:39,Washington at Kearney,46,395,Subscriber,94133
1,29181,115,9/20/2013 17:17,Post at Kearney,47,9/20/2013 17:19,Post at Kearney,47,345,Customer,
2,31420,156,9/23/2013 10:59,Post at Kearney,47,9/23/2013 11:01,Beale at Market,56,292,Subscriber,94115
3,97477,138,11/15/2013 12:54,Post at Kearney,47,11/15/2013 12:56,Market at Sansome,77,273,Subscriber,94107
4,115555,154,12/4/2013 19:00,Post at Kearney,47,12/4/2013 19:02,Washington at Kearney,46,314,Subscriber,94133
5,153585,141,1/14/2014 23:09,Post at Kearney,47,1/14/2014 23:11,Howard at 2nd,63,381,Subscriber,94105
6,188403,159,2/18/2014 18:43,Post at Kearney,47,2/18/2014 18:45,Washington at Kearney,46,631,Subscriber,94133


In [50]:
query = '''
SELECT duration FROM trip
ORDER BY duration DESC
LIMIT 1;
'''

run_query(query)

,duration
0,722236


In [51]:
query = '''
SELECT COUNT(*) AS "Count" 
FROM trip
WHERE end_station_name = '2nd at South Park'
'''

run_query(query)



,Count
0,2150


In [52]:
query = '''
SELECT COUNT(*) AS "Count" 
FROM trip
WHERE duration BETWEEN 1000 AND 2000
'''

run_query(query)


,Count
0,7225


In [56]:
query = '''
SELECT bike_id AS 'Bike', COUNT(*) AS 'Number of Trips'
FROM trip
GROUP BY bike_id
ORDER BY COUNT(*) DESC
limit 1;
'''

run_query(query)

,Bike,Number of Trips
0,392,277


In [58]:
query = '''
SELECT start_station_name, end_station_name, COUNT(*) AS 'Number of Round Trips'
FROM trip
WHERE start_station_name = end_station_name;
'''

run_query(query)

,start_station_name,end_station_name,Number of Round Trips
0,Redwood City Caltrain Station,Redwood City Caltrain Station,2951
